In [93]:
import importlib
import verse
import corpus
import phonetic
import generate_sample
import numpy as np

In [78]:
importlib.reload(phonetic)

dictionary = phonetic.PhoneticDictionary()

song_files = ["lyrics/kendrick_lamar/GoodKid.txt",
             "lyrics/cardi-b/BodakYellow.txt",
             "lyrics/nicki_minaj/ChunLi.txt",
             "lyrics/drake/NiceForWhat.txt",
             "lyrics/hamilton/processed/AlexanderHamilton.txt"]

unknowns = ["logios_output/kendrick_lamar/GoodKid_logios.txt",
           "logios_output/cardi-b/BodakYellow_logios.txt",
           "logios_output/nicki_minaj/ChunLi_logios.txt",
           "logios_output/drake/NiceForWhat_logios.txt",
           "logios_output/hamilton/AlexanderHamilton_logios.txt"]

kendrick_lamar_verse = "Mass hallucination baby \
Ill education baby \
Want to reconnect with your relations \
This is your station baby \
Look inside these walls and you see them having withdrawals \
Of a prisoner on his way \
Trapped inside your desire \
To fire bullets that stray \
Track a tire just tell you I'm tired and ran away \
I should ask a choir what do you require \
To sing a song that acquire me to have faith \
As the record spin I should pray \
For the record I recognize that I'm easily prey \
I got ate alive yesterday \
I got animosity building \
It's probably big as a building"

cardi_b_verse = "I'm the hottest in the street, know you prolly heard of me \
Got a bag and fixed my teeth, hope you hoes know it ain't cheap \
And I pay my mama bills, I ain't got no time to chill \
Think these hoes be mad at me, their baby father run a bill \
Said, Lil bitch, you can't fuck with me if you wanted to \
These expensive, these is red bottoms, these is bloody shoes \
Hit the store, I can get 'em both, I don't wanna choose \
And I'm quick, cut a nigga off, so don't get comfortable, look"

nicki_minaj_verse = "I went and copped the chopsticks \
Put it in my bun just to pop shit \
I'm always in the top shit \
Box seats, bitch, fuck the gossip \
How many a' them coulda did it with finnesse? \
Now everybody like, She really is the best. \
You play checkers, couldn't beat me playin' chess \
Now I'm about to turn around and beat my chest \
Bitch, it's King Kong, yes, it's King Kong \
Bitch, it's King Kong, this is King Kong \
Chinese ink on, Siamese links on \
Call me 2 Chainz, name go ding dong"

drake_verse = "Everybody get your mothafuckin' roll on \
I know shorty and she doesn't want no slow song \
Had a man last year, life goes on \
Haven't let that thing loose, girl, in so long \
You been inside, know you like to lay low \
I've been peepin' what you bringin' to the table \
Workin' hard, girl, everything paid for \
First, last phone bill, car note, cable \
With your phone out, gotta hit them angles \
With your phone out, snappin' like you Fabo \
And you showin' off, but it's alright \
And you showin' off, but it's alright \
It's a short life, yeah"

hamilton_verse = "How does a bastard, orphan, son of a whore and a \
Scotsman, dropped in the middle of a forgotten \
Spot in the Caribbean by Providence, impoverished, in squalor \
Grow up to be a hero and a scholar? \
The ten-dollar Founding Father without a father \
Got a lot farther by workin’ a lot harder \
By bein’ a lot smarter \
By bein’ a self-starter \
By fourteen, they placed him in charge of a trading charter"

verses = [kendrick_lamar_verse,
         cardi_b_verse,
         nicki_minaj_verse,
         drake_verse,
         hamilton_verse]

In [79]:
def make_template(text):
    template = verse.VerseTemplate(text, dictionary, breakrules="phrase")
    template.get_rhyme()
    return template

In [120]:
def compute_similarity_score(res1, res2):
    
    # iterate by syllable
    # check word at each syllable position and compare between strings
    
    # both templates should have the same number of syllables
    first_template, second_template = res1[1], res2[1]
    num_syls = first_template.num_syllables
    assert num_syls == second_template.num_syllables
    
    total_matches = 0
    for syl_pos in range(num_syls):
        word1 = first_template.wordList[first_template.matrix_indices[syl_pos][0]]
        word2 = second_template.wordList[second_template.matrix_indices[syl_pos][0]]
        # print(word1, word2)
        if word1 == word2:
            total_matches += 1
    print(total_matches, num_syls)
    return total_matches / num_syls

In [123]:
def intra_similarity(template_text, corpus, n_iters):
    
    results = []
    
    for i in range(n_iters):
        template = verse.VerseTemplate(template_text, dictionary,
                              breakrules='phrase')
        template.get_rhyme()
        result_text = generate_sample.fill_template(corp, template, verbose=False)
        print("\n", result_text)
        results.append((result_text, template))
        
    score_matrix = np.zeros((len(results), len(results)))
    for i in range(len(results)):
        for j in range(len(results)):
            score_matrix[i,j] = compute_similarity_score(results[i], results[j])
        
    print(score_matrix)

In [124]:
importlib.reload(corpus)
importlib.reload(verse)
importlib.reload(generate_sample)

corp = corpus.WordCorpus(dictionary)

for idx, song in enumerate(song_files):
    
    corp.initialize(song)
    corp.add_unknowns(unknowns[idx])
    corp.initializeMatrix()
    corp.initializeSylDict()

    intra_similarity(verses[idx], corp, 10)

    break
    


unknowns filename: lyrics/kendrick_lamar/GoodKid_unknowns.txt
Need to add 6 words to template word list using LOGIOS tool
Corpus text: 510 words, 252 unique

 me by myself promise me against my your station baby look for the blinking of them against my neck as a couple that acquire me to do you i'm tired and shrooms your station baby look inside your dog has to obtain the record i see is you can never me pick out a victim i see is your nike checks i recognize track the that look for morning i got time you wonder if i supposed for the silence in this is just reconnect with rank no better person i see is your station is just me in the worst side of your desire to persevere as a building me standing still i recognize that tooken  / 

 difference or cop on your desire to it's probably young but that acquire me against i heard this is racial profile i supposed to have faith as your dog persevere as a better person i know that look for the same society ate alive nothing faith as a function x

/Users/reedperkins/byu/cs673/generate_sample.py:119: RuntimeWarning: invalid value encountered in true_divide
  scansion_scores /= np.sum(scansion_scores)



 station baby want to paint than every time you hired me jumping off of my them every time to put me against my neck and these walls and shrooms affiliation mass hallucination baby want to hold me playing it a pleasure get blood on a better picture to to me playing it was like a pleasure in to put me you see is racial them candy i know that i'm tired and you ever me you hired me you i'm accustomed to just as the same society it's probably in this is me walking from bible study and these shrooms some grown up candy for the topic is just me standing still i got time  / 

 fire bullets that tooken place they was like that look for the top of a prisoner on tattoo of a song that i'm accustomed to persevere as a victim i see in my i supposed to put me through gang files but that acquire me walking from bible study and ate finding and me standing still i see is racial profile reconnect i got time you wonder ever to paint than me you hired me playing it was to fire bullets that look for the s